<img src="https://raw.githubusercontent.com/quantumlib/Cirq/master/docs/_static/Cirq_logo_color.png"  height="80px"/> [Thank you for the feedback!](https://forms.gle/oTG7xaT8zyq5svN88)

[Cirq](https://github.com/quantumlib/cirq) is a framework for writing quantum algorithms for noisy intermediate scale quantum (NISQ) devices. 

Roughly speaking, NISQ devices are those with O(100) qubits that can enact O(1000) gates.  Because the resources for NISQ devices are so constrained, we believe that a framework for writing programs on these devices needs to be aware of all of the architectural properties of the device on which the algorithm is written. This is in contrast to other frameworks where there is a clean separation between the abstract model being used and the details of the device.  

**In this notebook we'll run through some Cirq implementations of some of the standard algorithms that one encounters in an introductory quantum computing course. The discussion here is expanded from examples found in the [Cirq examples](https://github.com/quantumlib/Cirq/tree/master/examples) directory.**

**Instructions**: Go to File --> Save Copy in Drive to get your own copy to play with.

**Note:** Be aware that Cirq is still alpha software, meaning **we are still making breaking changes all the time**. If you don't want your project to suddenly go from working to not working when we release a new version, you should depend on a *specific version* of Cirq and periodically bump that version to the latest one. For the purposes of this tutorial, we will use version of `0.6` (i.e. `cirq==0.6` in pip's version notation).

>[Quantum Teleportation](#scrollTo=laCvAwThaADq)

>[The Deutsch-Jozsa Algorithm](#scrollTo=S8S_oxilOP03)

>>[The functions as oracles](#scrollTo=x7fHlB5gOxQN)

>>[Deutsch's algorithm](#scrollTo=1DIBPXcXJ7Dd)

>>[Two Bit Deutsch-Jozsa Algorithm](#scrollTo=IRrrWLbYKNIW)

>>[Exercise](#scrollTo=wKbP-31MKVtJ)

>>[Solution](#scrollTo=vInc4BMqKcbM)

>[Quantum Fourier Transform](#scrollTo=1fU1R-qjr1Oq)

>>[Overview](#scrollTo=GLwcGmq1r1On)

>>[Quantum Fourier Transform as a Circuit](#scrollTo=D7iWLL1Vr1Og)

>>>[Solution](#scrollTo=ZZhs4w58r1OY)

>>[Quantum Fourier Transform as a Gate](#scrollTo=hdxvyG4gr1OH)

>>>[Solution](#scrollTo=6EtrKjkpr1N9)

>>>[Test the Circuit](#scrollTo=LYHZdrL3r1Nz)

>>[Inverse QFT](#scrollTo=kc-0EeIar1Nn)

>>>[Solution](#scrollTo=ZUthaiCcr1Na)

>>[Is the QFT_inv gate the inverse of the QFT Circuit?](#scrollTo=KxgoU3L6r1NO)

>>>[Solution](#scrollTo=OPp9BNvSr1NJ)

>[Phase Estimation](#scrollTo=r-CjbPwkRI_I)

>>[Exercise: As a function](#scrollTo=Dn9j1RCrpCj_)

>>>[Solution](#scrollTo=5cf9gaXCpmq4)

>>[Phase Estimation Without an Eigenstate](#scrollTo=cr-NVLG2loo7)

>>>[Solution](#scrollTo=pnddp79QmZAl)

>[Grover's Algorithm](#scrollTo=ksA-fvrZaT5g)

>>[Grover on the custom Device](#scrollTo=2vDffuikV9Fb)

>[Left to the reader as an exercise](#scrollTo=c8Zf-MRDPynh)

>>[Quantum Fourier Transform with Unreversed Output](#scrollTo=zQzAYK-VNVDu)

>>[Phase Estimation with Arbitrary $U$](#scrollTo=EG6cDFWJk-ZS)

>>[QFT and Phase Estimation with Adjacency Constraints](#scrollTo=9WKZoaav0dtt)



In [ ]:
# install cirq
!pip install cirq==0.6 --quiet

     |████████████████████████████████| 1.2MB 2.8MB/s 
     |████████████████████████████████| 1.8MB 47.8MB/s 
     |████████████████████████████████| 1.2MB 34.5MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.


To verify that Cirq is installed in your environment, try to `import cirq` and print out a diagram of the Bristlecone device.

In [ ]:
import cirq
import numpy as np
import random

# How many qubits are in the Bristlecone device?
print(len(cirq.google.Bristlecone.qubits))

72


# Quantum Teleportation

Quantum Teleportation is a process by which a quantum state can be transmitted
by sending only two classical bits of information. This is accomplished by
pre-sharing an entangled state between the sender (Alice) and the receiver
(Bob). This entangled state allows the receiver (Bob) of the two classical
bits of information to possess a qubit with the same state as the one held by
the sender (Alice).

In the following example output, qubit 0 (the Message) is set to a random state
by applying X and Y gates. By sending two classical bits of information after
qubit 0 (the Message) and qubit 1 (Alice's entangled qubit) are measured, the
final state of qubit 2 (Bob's entangled qubit) will be identical to the
original random state of qubit 0 (the Message). This is only possible given
that an entangled state is pre-shared between Alice and Bob.

=== REFERENCES ===

https://en.wikipedia.org/wiki/Quantum_teleportation
https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.70.1895

In [ ]:
def make_quantum_teleportation_circuit(gate):
    circuit = cirq.Circuit()
    msg, alice, bob = cirq.LineQubit.range(3)

    # Creates Bell state to be shared between Alice and Bob
    circuit.append([cirq.H(alice), cirq.CNOT(alice, bob)])
    # Creates a random state for the Message
    circuit.append(gate(msg))
    
    # Bell measurement of the Message and Alice's entangled qubit
    circuit.append([cirq.CNOT(msg, alice), cirq.H(msg)])
    circuit.append(cirq.measure(msg, alice))
    
    # Uses the two classical bits from the Bell measurement to recover the
    # original quantum Message on Bob's entangled qubit
    circuit.append([cirq.CNOT(alice, bob), cirq.CZ(msg, bob)])

    return circuit


gate = cirq.SingleQubitMatrixGate(cirq.testing.random_unitary(2))
circuit = make_quantum_teleportation_circuit(gate)

print("Circuit:")
print(circuit)

sim = cirq.Simulator()

# Create qubits.
q0 = cirq.LineQubit(4)

# Produces the message using random unitary
message = sim.simulate(cirq.Circuit([gate(q0)]))

print("Bloch Vector of Message After Random Unitary:")
# Prints the Bloch vector of the Message after the random gate
b0X, b0Y, b0Z = cirq.bloch_vector_from_state_vector(
    message.final_state, 0)
print("x: ", np.around(b0X, 4),
      "y: ", np.around(b0Y, 4),
      "z: ", np.around(b0Z, 4))

# Records the final state of the simulation
final_results = sim.simulate(circuit)

print("Bloch Sphere of Qubit 2 at Final State:")
# Prints the Bloch Sphere of Bob's entangled qubit at the final state
b2X, b2Y, b2Z = cirq.bloch_vector_from_state_vector(
    final_results.final_state, 2)

print("x: ", np.around(b2X, 4),
      "y: ", np.around(b2Y, 4),
      "z: ", np.around(b2Z, 4))

Circuit:
      ┌                           ┐
0: ───│-0.524-0.42j  -0.221+0.707j│───────@───H───M───────@───
      │-0.361-0.647j  0.427-0.518j│       │       │       │
      └                           ┘       │       │       │
                                          │       │       │
1: ───H───────────────────────────────@───X───────M───@───┼───
                                      │               │   │
2: ───────────────────────────────────X───────────────X───@───
Bloch Vector of Message After Random Unitary:
x:  0.9218 y:  0.3748 z:  -0.0988
Bloch Sphere of Qubit 2 at Final State:
x:  0.9218 y:  0.3748 z:  -0.0988


# The Deutsch-Jozsa Algorithm

The very first indication that quantum computers could be more powerful than classical computers was provided by David Deutsch in his 1985 paper:
> David Deutsch,  "Quantum Theory, the Church-Turing Principle and the Universal Quantum Computer" Proceedings of the Royal Society of London A. 400: 97 [PDF](https://people.eecs.berkeley.edu/~christos/classics/Deutsch_quantum_theory.pdf)

This algorithm was extended by Deutsch and Richard Jozsa to a more convincing algorithmic separation and what is now called the Deutsch-Jozsa algorithm.  In this section we will show how to write circuits for the Deutsch algorithm.

Then as an exercise in using Cirq for algorithms for a small version of the Deutsch-Jozsa algorithm. Let's begin with the Deutsch algorithm. 


In Deutsch's algorithm you are given access to a box which computes a one bit boolean function.  That is it is a box which takes in a bit and outputs a bit.  If we want to be a mathematician or theoretical computer scientist we write the function $f$ as $f: \{0, 1\} \rightarrow \{0, 1\}$.  There are exactly four such boolean functions which we can write out in a table

| $x$ || $f_0$ | $f_1$ | $f_x$ | $f_{\bar{x}}$ |
| --- ||  --- | --- | --- | --- |
| 0 || 0 | 1 | 0 | 1
| 1 || 0 | 1 | 1 | 0

* The first two functions ($f_0$ and $f_1$) are *constant* . That is they always output a constant value. 
* The other two functions $f_x$ and $f_\bar{x}$ are *balanced*.  Over their inputs $0$ and $1$, they have an equal number of $0$s and $1$s in their truth table.  

We can now state Deutsch's problem:

> Given access to a one bit input one bit output boolean function, determine, by querying the function whether the function is *balanced* or *constant*.

It shouldn't take you much to convince yourself that in order to solve this problem classically you need to call the function on both possible input values.  The easy way to see this is just to consider what happens if you query the function on one particular input and notice that for either input learning the value of the function does not separate the constant from balanced functions.

**Classically one must query the binary function twice to distinguish the constant function from the balanced function.**




## The functions as oracles
Now lets turn to the quantum approach to this problem.  **There is one bit of book keeping we need to take care of: f(x)**.  Above we have described a classical function on bits that is not reversible.  That is, knowing the values of the output does not allow us to determine uniquely the value of the input.  In order to run this on a quantum computer, however we need to make this computation reversible.  A trick for taking a classical non-reversible function and making it quantum-compatible is to compute the value in an extra register. 

**Note**: Such extra registers are often called ancilla.

Suppose we have an $n$ bit input $x$ and we are computing a (potentially non-reverisble) boolean function $f(x)$.  Then we can implement this via a Unitary $U_f$ that acts on $n + 1$ qubits

$$U_f |x\rangle |y\rangle = |x\rangle | y \oplus f(x)\rangle$$

Here $\oplus$ is addition modulo $2$ (XOR) and we have identified how $U_f$ acts by its action on all computational basis states $|x\rangle$ ($n$ input qubits) and ($1$ output qubit)$|y\rangle$. To see that this is reversible one can note that applying the transformation twice returns the state to its original form.

**Example**: Let's see how to implement these functions in Cirq, in the case that $n=1$, thus for two qubits.

$f_0$ (always zero function) enacts the transform
$$
\begin{eqnarray}
|00\rangle &\rightarrow&  |00\rangle \\
|01\rangle &\rightarrow&  |01\rangle \\
|10\rangle &\rightarrow&  |10\rangle \\
|11\rangle &\rightarrow&  |11\rangle \\
\end{eqnarray}
$$
Well this is just the identity transform on the second qubit, i.e. an empty circuit.

$f_1$ (always one function) enacts the transform
$$
\begin{eqnarray}
|00\rangle &\rightarrow&  |01\rangle \\
|01\rangle &\rightarrow&  |00\rangle \\
|10\rangle &\rightarrow&  |11\rangle \\
|11\rangle &\rightarrow&  |10\rangle \\
\end{eqnarray}
$$
This is the `cirq.X` bit flip gate on the second qubit.

$f_x$ (identity function $f_x = x$) enacts the transform
$$
\begin{eqnarray}
|00\rangle &\rightarrow&  |00\rangle \\
|01\rangle &\rightarrow&  |01\rangle \\
|10\rangle &\rightarrow&  |11\rangle \\
|11\rangle &\rightarrow&  |10\rangle \\
\end{eqnarray}
$$
This is nothing more than a `cirq.CNOT` from the first bit to the second bit.

Finally $f_\bar{x}$ enacts the transform
$$
\begin{eqnarray}
|00\rangle &\rightarrow&  |01\rangle \\
|01\rangle &\rightarrow&  |00\rangle \\
|10\rangle &\rightarrow&  |10\rangle \\
|11\rangle &\rightarrow&  |11\rangle \\
\end{eqnarray}
$$
which is a `cirq.CNOT` from the first bit to the second bit followed by a `cirq.X` on the second bit.

We can encapulate these functions into a dictionary from a oracle name to the quantum operations (gates) in the circuit needed to enact this function on the ancilla qubit:

In [ ]:
# The qubits
q0, q1 = cirq.LineQubit.range(2)

# The functions as oracles
oracles = {
    '0': [],
    '1': [cirq.X(q1)],
    'x': [cirq.CNOT(q0, q1)],
    'notx': [cirq.CNOT(q0, q1), cirq.X(q1)]
}    

## Deutsch's algorithm

Suppose we are given access to the reversible oracle functions we have defined before.  By a similar argument for our irreversible classical functions you can show that you cannot distinguish the balanced from the constant functions by using this oracle only once.  But now we can ask the question: what if we are allowed to query this box in superposition, i.e. what if we can use the power of quantum computing?

Deutsch was able to show that you could solve this problem now, with quantum computers, using only a single query of the box.  To see how this works we need two simple insights.

Suppose that we prepare the second qubit in the superposition state $|-\rangle=\frac{1}{\sqrt{2}}(|0\rangle-|1\rangle)$ and apply  the oracle.  Then we can check that
$$ 
U_f |x\rangle |-\rangle = U_f|x\rangle \frac{1}{\sqrt{2}}(|0\rangle -|1\rangle ) = |x\rangle \frac{1}{\sqrt{2}}(|f(x)\rangle -|f(x) \oplus 1\rangle ) =  (-1)^{f(x)} |x\rangle |-\rangle
$$  
**This is the so called "phase kickback trick".  By applying $U_f$ onto a target which is in superposition, the value of the function ends up showing up in the global phase.**  

How can we leverage this to distinguish between the constant and balanced functions?  Note that for the constant functions the phase that is applied is the same for all inputs $|x\rangle$, whereas for the balanced functions the phase is different for each value of $x$.   Another way to say that is that if we use the phase kickback trick then for each of the oracles we apply the following transform on the first qubit.
$$
\begin{eqnarray}
f_0 \rightarrow I, &&
f_1 \rightarrow -I, &&
f_x \rightarrow Z, &&
f_\bar{x} \rightarrow -Z &&
\end{eqnarray}
$$
Now we only need, on the first qubit, to distinguish between the identity gate and the $Z$ gate.  But we can do this by recalling the identity
$$ H Z H = X$$
where $H$ is the Hamadard gate
$$H = {1 \over \sqrt{2}} \left[ \begin{array}[cc]  & 1 & 1  \\ 1 & -1 \end{array}\right]$$

**This means that we can turn a phase flip into a bit flip by applying Hadamards before and after the phase flip.**


If we look at the constant and balanced functions we see that this means that the constant functions will be proportional to $I$ and the balanced will be proportional to $X$.  If we feed in $|0\rangle$ to this register, then in the first cases we will only see $|0\rangle$ and in the second case we will see $|1\rangle$.  In other words we will be able to distinguish constant from balanced using a single query of the oracle.


Let's code this up


In [ ]:
# q0, q1 = cirq.LineQubit.range(2)
# oracles = {
#     '0': [],
#     '1': [cirq.X(q1)],
#     'x': [cirq.CNOT(q0, q1)],
#     'notx': [cirq.CNOT(q0, q1), cirq.X(q1)]
# }    

def deutsch_algorithm(oracle):
    yield cirq.X(q1)
    yield cirq.H(q0), cirq.H(q1)
    yield oracle
    yield cirq.H(q0)
    yield cirq.measure(q0)

for key, oracle in oracles.items():
    print('Circuit for {}...'.format(key))
    print('{}\n'.format(cirq.Circuit(deutsch_algorithm(oracle))))

Circuit for 0...
0: ───H───H───M───

1: ───X───H───────

Circuit for 1...
0: ───H───H───M───

1: ───X───H───X───

Circuit for x...
0: ───H───────@───H───M───
              │
1: ───X───H───X───────────

Circuit for notx...
0: ───H───────@───H───M───
              │
1: ───X───H───X───X───────



Lets run these circuits a bunch of times to see that the measurement result ends up correctly distinguishing constant from balanced.

In [ ]:
simulator = cirq.Simulator()

for key, oracle in oracles.items():
    result = simulator.run(cirq.Circuit(deutsch_algorithm(oracle)), 
                          repetitions=10)
    print('oracle: {:<4} results: {}'.format(key, result))

oracle: 0    results: 0=0000000000
oracle: 1    results: 0=0000000000
oracle: x    results: 0=1111111111
oracle: notx results: 0=1111111111


##Two Bit Deutsch-Jozsa Algorithm

All Boolean functions for one input bit are either constant or balanced.

For boolean functions from two input bits not all functions are constant or balanced.
* There are two constant functions, $f(x_0, x_1) = 0$ and $f(x_0, x_1)=1$, 
* There are ${4 \choose 2} = 6$ balanced functions. 

The following code gives you the operations for these functions where we take two input qubits and compute the function in the third qubit

In [ ]:
# Three qubits for two bit functions
q0, q1, q2 = cirq.LineQubit.range(3)

# These are the functions
# Output is always 0 and Outputs is always 1
constant = ([], [cirq.X(q2)])
# Four functions
balanced = ([cirq.CNOT(q0, q2)], [cirq.CNOT(q1, q2)], [cirq.CNOT(q0, q2), cirq.CNOT(q1, q2)],
            [cirq.CNOT(q0, q2), cirq.X(q2)], [ cirq.CNOT(q1, q2), cirq.X(q2)], [cirq.CNOT(q0, q2), cirq.CNOT(q1, q2), cirq.X(q2)])

for i, ops in enumerate(constant):
    print('\nConstant function {}'.format(i))
    print(cirq.Circuit(ops).to_text_diagram(qubit_order=[q0, q1, q2]))
    print()

for i, ops in enumerate(balanced):
    print('\nBalanced function {}'.format(i))
    print(cirq.Circuit(ops).to_text_diagram(qubit_order=[q0, q1, q2]))


Constant function 0
0: ───

1: ───

2: ───


Constant function 1
0: ───────

1: ───────

2: ───X───


Balanced function 0
0: ───@───
      │
1: ───┼───
      │
2: ───X───

Balanced function 1
0: ───────

1: ───@───
      │
2: ───X───

Balanced function 2
0: ───@───────
      │
1: ───┼───@───
      │   │
2: ───X───X───

Balanced function 3
0: ───@───────
      │
1: ───┼───────
      │
2: ───X───X───

Balanced function 4
0: ───────────

1: ───@───────
      │
2: ───X───X───

Balanced function 5
0: ───@───────────
      │
1: ───┼───@───────
      │   │
2: ───X───X───X───


##Exercise

An extension of Deutsch's orginal algorithm is the Deutsch-Jozsa algorithm, which can distinguish constant from balanced functions like these using a single query to the oracle.  Write a quantum circuit that can distinguish these 

In [ ]:
simulator = cirq.Simulator()
def your_circuit(oracle):
    """
      your code here
    """
    
    yield oracle
    
    """
      your code here
    """
    
    yield cirq.measure(q0, q1, q2)

print('Your result on constant functions')
for oracle in constant:
    result = simulator.run(cirq.Circuit(your_circuit(oracle)), repetitions=10)
    print(result)
    
print('Your result on balanced functions')
for oracle in balanced:
    result = simulator.run(cirq.Circuit(your_circuit(oracle)), repetitions=10)
    print(result)

Your result on constant functions
0,1,2=0000000000, 0000000000, 0000000000
0,1,2=0000000000, 0000000000, 1111111111
Your result on balanced functions
0,1,2=0000000000, 0000000000, 0000000000
0,1,2=0000000000, 0000000000, 0000000000
0,1,2=0000000000, 0000000000, 0000000000
0,1,2=0000000000, 0000000000, 1111111111
0,1,2=0000000000, 0000000000, 1111111111
0,1,2=0000000000, 0000000000, 1111111111


##Solution

In [ ]:
# The generator for the circuit
def your_circuit(oracle):
    # phase kickback trick
    # Initialise in |->
    yield cirq.X(q2), cirq.H(q2)
    
    # equal superposition over input bits
    yield cirq.H(q0), cirq.H(q1)
    
    # query the function
    yield oracle
    
    # interference to get result, put last qubit into |1>
    yield cirq.H(q0), cirq.H(q1), cirq.H(q2)
    
    # 
    # Do postprocessing of the measurement results using quantum gates
    # 
    # A final OR gate to put result in final qubit
    # CCX is Toffoli gate which computes on the last bit
    # XOR(q2, AND (q0, q1)) which is equivalent to NAND(q0, q1)
    # If one negates the inputs to Toffoli, the results are
    #   If constant q2 is 0
    #   If balanced q2 is 1
    yield cirq.X(q0), cirq.X(q1), cirq.CCX(q0, q1, q2)
    
    yield cirq.measure(q0, q1, q2)


# Simulator
repetitions = 10
simulator = cirq.Simulator()


print('Your result on constant functions')
for oracle in constant:
    result = simulator.run(cirq.Circuit(your_circuit(oracle)), repetitions=repetitions)
    print(result)
    print(result.histogram(key="0,1,2"))
    
print('Your result on balanced functions')
for oracle in balanced:
    result = simulator.run(cirq.Circuit(your_circuit(oracle)), repetitions=repetitions)
    # Measurements are 0,1,2 -> key is "0,1,2"
    print(result)
    print(result.histogram(key="0,1,2"))

Your result on constant functions
0,1,2=1111111111, 1111111111, 0000000000
Counter({6: 10})
0,1,2=1111111111, 1111111111, 0000000000
Counter({6: 10})
Your result on balanced functions
0,1,2=0000000000, 1111111111, 1111111111
Counter({3: 10})
0,1,2=1111111111, 0000000000, 1111111111
Counter({5: 10})
0,1,2=0000000000, 0000000000, 1111111111
Counter({1: 10})
0,1,2=0000000000, 1111111111, 1111111111
Counter({3: 10})
0,1,2=1111111111, 0000000000, 1111111111
Counter({5: 10})
0,1,2=0000000000, 0000000000, 1111111111
Counter({1: 10})


# Quantum Fourier Transform

This section provides an overview of the quantum Fourier transform and its use in the Phase Estimation algorithm in Cirq.

## Overview

We'll start out by reminding ourselves what the [quantum Fourier transform](https://en.wikipedia.org/wiki/Quantum_Fourier_transform) does, and how it should be constructed.

Suppose we have $n$ qubits in the state $|x\rangle$, where $x$ is an integer in the range $0$ to $2^{n-1}$. The quantum Fourier transform (QFT) performs the following operation:
$$
\text{QFT}|x\rangle = \frac{1}{2^{n/2}} \sum_{y=0}^{2^n-1} e^{2\pi i y x/2^n} |y\rangle.
$$
When $x=0$, this is the same as the action of $H^{\otimes n}$ (this is an important sanity check). Though it may not be obvious at first glance, the QFT is actually a unitary transformation. As a matrix, the QFT is given by
$$
\text{QFT} = \begin{bmatrix}
1 & 1 & 1& \cdots &1 \\
1 & \omega & \omega^2& \cdots &\omega^{2^n-1} \\
1 & \omega^2 & \omega^4& \cdots &\omega^{2(2^n-1)}\\
\vdots &\vdots &\vdots &\ddots &\vdots \\
1 &\omega^{2^n-1} &\omega^{2(2^n-1)} &\cdots &\omega^{(2^n-1)(2^n-1)},
\end{bmatrix}
$$
where $\omega = e^{2\pi i /2^n}$. If you believe that the QFT is unitary, then you'll also notice from the matrix form that its inverse is given by a similar expression but with complex-conjugated coefficients:
$$
\text{QFT}^{-1}|x\rangle = \frac{1}{2^{n/2}} \sum_{y=0}^{2^n-1} e^{-2\pi i y x/2^n} |y\rangle.
$$

The construction of the QFT as a circuit follows a simple recursive form, though fully justifying it will take us too far from the main goal of this notebook. We really only need to know what the circuit looks like, and for that we can look at this picture from the Wikipedia article:

>![QFT Circuit](https://upload.wikimedia.org/wikipedia/commons/6/61/Q_fourier_nqubits.png)

We just need to understand the notation a little bit. $x_j$ on the left-hand side represents one of the binary digits of the input $x$. $x_1$ is the most significant bit and $x_n$ the least significant bit:
$$
x = \sum_{j=0}^{n-1} x_{j+1}2^j.
$$
$H$ is the Hadamard gate, as usual. The Controlled-$R_j$ gates are phase gates similar to the Controlled-$Z$ gate. In fact, for us it will be useful to just think of them as fractional powers of Controlled-$Z$ gates:
$$
CR_j = CZ^{\large 1/2^{j-1}}
$$
Finally, on the far right we have the output representing the bts of $y$. The only difference between the left and right side is that the output bits are in a different order: the most significant bit of $y$ is on the bottom and the least significant bit is on the top.

## Quantum Fourier Transform as a Circuit

Let's define a generator which produces the QFT circuit. It should accept a list of qubits as input and `yield`s the gates to construct the QFT in the right order. A useful observation is that the the QFT circuit "repeats" smaller versions of itself as you move from left to right across the diagram.

In [ ]:
def make_qft(qubits):
  """Generator for the QFT on an arbitrary number of qubits. With four qubits
  the answer is
  ---H--@-------@--------@---------------------------------------------
        |       |        |
  ------@^0.5---+--------+---------H--@-------@------------------------
                |        |            |       |
  --------------@^0.25---+------------@^0.5---+---------H--@-----------
                         |                    |            |
  -----------------------@^0.125--------------@^0.25-------@^0.5---H---
  """
  # YOUR CODE HERE

### Solution

In [ ]:
def make_qft(qubits):
  """Generator for the QFT on an arbitrary number of qubits. With four qubits
  the answer is
  ---H--@-------@--------@---------------------------------------------
        |       |        |
  ------@^0.5---+--------+---------H--@-------@------------------------
                |        |            |       |
  --------------@^0.25---+------------@^0.5---+---------H--@-----------
                         |                    |            |
  -----------------------@^0.125--------------@^0.25-------@^0.5---H---
  """
  qubits = list(qubits)
  while len(qubits) > 0:
      q_head = qubits.pop(0)
      yield cirq.H(q_head)
      for i, qubit in enumerate(qubits):
          yield (cirq.CZ**(1/2**(i+1)))(qubit, q_head)

In [ ]:
num_qubits = 4
qubits = cirq.LineQubit.range(num_qubits)

qft = cirq.Circuit(make_qft(qubits))
print(qft)

                  ┌───────┐   ┌────────────┐   ┌───────┐
0: ───H───@────────@───────────@───────────────────────────────────────
          │        │           │
1: ───────@^0.5────┼─────H─────┼──────@─────────@──────────────────────
                   │           │      │         │
2: ────────────────@^0.25──────┼──────@^0.5─────┼─────H────@───────────
                               │                │          │
3: ────────────────────────────@^(1/8)──────────@^0.25─────@^0.5───H───
                  └───────┘   └────────────┘   └───────┘


## Quantum Fourier Transform as a Gate



For later convenience, it will be useful to encapsulate the QFT construction into a single gate. We can inherit from  `cirq.Gate` to define a gate which acts on an unspecified number of qubits, and then use the same strategy as for `make_qft` in the `_decompose_` method of the gate. Fill in the following code block to make a QFT gate.

In [ ]:
class QFT(cirq.Gate):
  """Gate for the Quantum Fourier Transformation
  """
  
  def __init__(self, n_qubits):
    self.n_qubits = n_qubits

  def num_qubits(self):
    return self.n_qubits    
    
#   def _decompose_(self, qubits):
  # YOUR CODE HERE
            
  # How should the gate look in ASCII diagrams?          
  def _circuit_diagram_info_(self, args):        
    return tuple('QFT{}'.format(i) for i in range(self.n_qubits))

### Solution

A copy/paste is all that's required here:

In [ ]:
class QFT(cirq.Gate):
  """Gate for the Quantum Fourier Transformation
  """
  
  def __init__(self, n_qubits):
    self.n_qubits = n_qubits

  def num_qubits(self):
    return self.n_qubits
    
  def _decompose_(self, qubits):
    """Implements the QFT on an arbitrary number of qubits. The circuit
    for num_qubits = 4 is given by
    ---H--@-------@--------@---------------------------------------------
          |       |        |
    ------@^0.5---+--------+---------H--@-------@------------------------
                  |        |            |       |
    --------------@^0.25---+------------@^0.5---+---------H--@-----------
                           |                    |            |
    -----------------------@^0.125--------------@^0.25-------@^0.5---H---
    """
    qubits = list(qubits)
    while len(qubits) > 0:
        q_head = qubits.pop(0)
        yield cirq.H(q_head)
        for i, qubit in enumerate(qubits):
            yield (cirq.CZ**(1/2**(i+1)))(qubit, q_head)
            
  # How should the gate look in ASCII diagrams?          
  def _circuit_diagram_info_(self, args):        
    return tuple('QFT{}'.format(i) for i in range(self.n_qubits))

### Test the Circuit

We should confirm that the gate we've defined is actually doing the same thing as the `make_qft` function from before. We can do that with the following test:

In [ ]:
num_qubits = 4

qubits = cirq.LineQubit.range(num_qubits)
circuit = cirq.Circuit(QFT(num_qubits).on(*qubits))
print(circuit)

qft_test = cirq.Circuit(make_qft(qubits))
print(qft_test)
np.testing.assert_allclose(cirq.unitary(qft_test), cirq.unitary(circuit))

0: ───QFT0───
      │
1: ───QFT1───
      │
2: ───QFT2───
      │
3: ───QFT3───
                  ┌───────┐   ┌────────────┐   ┌───────┐
0: ───H───@────────@───────────@───────────────────────────────────────
          │        │           │
1: ───────@^0.5────┼─────H─────┼──────@─────────@──────────────────────
                   │           │      │         │
2: ────────────────@^0.25──────┼──────@^0.5─────┼─────H────@───────────
                               │                │          │
3: ────────────────────────────@^(1/8)──────────@^0.25─────@^0.5───H───
                  └───────┘   └────────────┘   └───────┘


## Inverse QFT

We also want to implement the inverse QFT, which we'll do with a completely separate gate. Modify the `QFT` gate from above to create a `QFT_inv` gate:

In [ ]:
class QFT_inv(cirq.Gate):
  """Gate for the inverse Quantum Fourier Transformation
  """
  
  # YOUR CODE HERE

### Solution

Compared to the `QFT` code above, we just have to add in a few minus signs.

You can convince yourself that this is the same as complex-conjugating the associated unitary matrix of the QFT, which gives us the inverse QFT.

In [ ]:
class QFT_inv(cirq.Gate):
  """Gate for the inverse Quantum Fourier Transformation
  """
  
  def __init__(self, n_qubits):
    self.n_qubits = n_qubits

  def num_qubits(self):
    return self.n_qubits   
    
  def _decompose_(self, qubits):
    """Implements the inverse QFT on an arbitrary number of qubits. The circuit
    for num_qubits = 4 is given by
    ---H--@-------@--------@---------------------------------------------
          |       |        |
    ------@^-0.5--+--------+---------H--@-------@------------------------
                  |        |            |       |
    --------------@^-0.25--+------------@^-0.5--+---------H--@-----------
                           |                    |            |
    -----------------------@^-0.125-------------@^-0.25------@^-0.5--H---
    """
    qubits = list(qubits)
    while len(qubits) > 0:
        q_head = qubits.pop(0)
        yield cirq.H(q_head)
        for i, qubit in enumerate(qubits):
            yield (cirq.CZ**(-1/2**(i+1)))(qubit, q_head)
                      
  def _circuit_diagram_info_(self, args):        
    return tuple('QFT{}^-1'.format(i) for i in range(self.n_qubits))

## Is the QFT_inv gate the inverse of the QFT Circuit?

We chose not to define the `QFT_inv` as literally the inverse of the `QFT` gate. Why was that? What would you get if you concetenated the `QFT` and `QFT_inv` gates as defined? (Try it!) Can you put together `QFT` and `QFT_inv` in such a way that one undoes the action of the other? This exercise doubles as a test of the `QFT_inv` implementation.

### Solution

The QFT and QFT_inv circuits we have defined are not literal inverses of each other because the both reverse the order of the bits when going from input to output. We can explicitly see this in the following code block:

In [ ]:
num_qubits = 2

qubits = cirq.LineQubit.range(num_qubits)
circuit = cirq.Circuit(QFT(num_qubits).on(*qubits),
                      QFT_inv(num_qubits).on(*qubits))
print(circuit)
print()
cirq.unitary(circuit).round(2)

0: ───QFT0───QFT0^-1───
      │      │
1: ───QFT1───QFT1^-1───



array([[ 1. +0.j ,  0. +0.j ,  0. +0.j ,  0. +0.j ],
       [-0. +0.j ,  0.5+0.5j, -0. +0.j ,  0.5-0.5j],
       [ 0. +0.j ,  0.5+0.j ,  0.5-0.5j,  0. +0.5j],
       [-0. +0.j ,  0. -0.5j,  0.5+0.5j,  0.5+0.j ]])

If `QFT` and `QFT_inv` were really inverses then we would have gotten the identity matrix here.

There are a couple of ways to fix this. 
* A: Change the implementations of these two gates in such a way that the outputs are "rightside-up"
* B: Turn the qubits around in between acting with `QFT` and `QFT_inv`.

The second solution is chosen, in the following.
In other words, we can insert the `QFT_inv` gate "upside-down" as follows:

In [ ]:
num_qubits = 2

qubits = cirq.LineQubit.range(num_qubits)
circuit = cirq.Circuit(QFT(num_qubits).on(*qubits),
                       QFT_inv(num_qubits).on(*qubits[::-1])) # qubit order reversed
print(circuit)
print()
cirq.unitary(circuit)

0: ───QFT0───QFT1^-1───
      │      │
1: ───QFT1───QFT0^-1───



array([[ 1.+0.j,  0.+0.j,  0.+0.j,  0.+0.j],
       [ 0.+0.j,  1.+0.j,  0.+0.j,  0.+0.j],
       [-0.+0.j, -0.+0.j,  1.+0.j, -0.+0.j],
       [-0.+0.j, -0.+0.j, -0.+0.j,  1.+0.j]])

We find the identity matrix, as desired (up to finite-precision numerical errors). Notice that the `QFT_inv` gate is upside-down relative to the `QFT` gate in the diagram. This is why we included the extra digits in the `wire_symobls`!

# Phase Estimation

We'll implement the [phase estimation](https://en.wikipedia.org/wiki/Quantum_phase_estimation_algorithm) algorithm.It uses the inverse QFT to give an estimate of the eigenvalue of a unitary operator. The total circuit that we are going to implement is given by

>![Phase Estimation](https://upload.wikimedia.org/wikipedia/commons/a/a5/PhaseCircuit-crop.svg)

Suppose we have a unitary operator $U$ with eigenvactor $|\psi\rangle$ and eigenvalue $\exp(2\pi i \theta)$. Our objective is to get an $n$-bit approximation to $\theta$. 

**The first step** is to construct the state
$$
|\Phi\rangle = \frac{1}{2^{n/2}}\sum_{y=0}^{2^{n-1}} e^{2\pi i y \theta}|y\rangle.
$$
This looks very similar to the output of the QFT applied to the state $|2^n\theta\rangle$, except for the fact that $2^n\theta$ may not be an integer. If $2^n\theta$ *were* an integer, then we would apply the inverse QFT and measure the qubits to read off the binary representation of $2^n\theta$. Even if $2^n\theta$ is not an integer, we can still perform the same procedure and the result will be a sequence of bits that, with high probility, gives an $n$-bit approximation to $\theta$. We just have to repeat the procedure a few times to be sure of the answer.

Since we've already constructed the inverse QFT, all we really have to do is figure out how to construct the magic state $|\Phi\rangle$. This is accomplished by the first part of the circuit pictured above. We begin by applying $H^{\otimes n}$ to the state $|0\rangle$, creating an equal superposition over all basis states:
$$
H^{\otimes n} |0\rangle = \frac{1}{2^{n/2}}\sum_{y=0}^{2^n-1}|y\rangle.
$$

**The second step** is to insert the correct phase coefficients. This is done by a sequence of Controlled-$U^k$ operations, where the qubits of $y$ are the controls and the $U^k$ operations act on $|\psi \rangle$. Let's try to implement this part of the procedure in Cirq, and then put it together with our `QFT_inv` from above.

For the gate $U$ we'll pick the single-qubit operation for $\theta \in [0,1)$. This is just for simplicity and ease of testing.
$$
U = Z^{2\theta} = \begin{bmatrix}
1 & 0 \\
0 & e^{2\pi i \theta }
\end{bmatrix}
$$


Later, you are invited to write an implementation that accepts an arbitrary $U$.

In [ ]:
theta = 0.234 # Try your own
n_bits = 3 # Accuracy of the estimate for theta. Try different values.

qubits = cirq.LineQubit.range(n_bits)
u_bit = cirq.NamedQubit('u')

U = cirq.Z**(2*theta)

phase_estimator = cirq.Circuit()

phase_estimator.append(cirq.H.on_each(*qubits))

for i, bit in enumerate(qubits):
  phase_estimator.append(cirq.ControlledGate(U).on(bit,u_bit)**(2**(n_bits-1-i)))

print(phase_estimator)

0: ───H───@──────────────────────────────
          │
1: ───H───┼──────────@───────────────────
          │          │
2: ───H───┼──────────┼─────────@─────────
          │          │         │
u: ───────Z^-0.128───Z^0.936───Z^0.468───


**The third step** is to perform the inverse QFT on estimation qubits:

In [ ]:
phase_estimator.append(QFT_inv(n_bits).on(*qubits))
print(phase_estimator)

0: ───H───@──────────────────────────────QFT0^-1───
          │                              │
1: ───H───┼──────────@───────────────────QFT1^-1───
          │          │                   │
2: ───H───┼──────────┼─────────@─────────QFT2^-1───
          │          │         │
u: ───────Z^-0.128───Z^0.936───Z^0.468─────────────


**Almost done**, before measuring the estimation qubits. But we should also specify an initial state for the `u_bit`. By default it will be the state $|0\rangle$, but the phase for that state is trivial with the operator we chose. Inserting a Pauli $X$ operator at the begining of the circuit changes this to the $|1\rangle$ state, which has the nontrivial $\theta$ phase. 

In [ ]:
# Add measurements to the end of the circuit
phase_estimator.append(cirq.measure(*qubits, key='m'))

# Add gate to change initial state to |1>
# Very similar to Deutsch initialization of last qubit
phase_estimator.insert(0,cirq.X(u_bit))

print(phase_estimator)

0: ───H───@──────────────────────────────QFT0^-1───M('m')───
          │                              │         │
1: ───H───┼──────────@───────────────────QFT1^-1───M────────
          │          │                   │         │
2: ───H───┼──────────┼─────────@─────────QFT2^-1───M────────
          │          │         │
u: ───X───Z^-0.128───Z^0.936───Z^0.468──────────────────────


**Simulate with Cirq** Now we can instantiate a simulator and make measurements of the estimation qubits. Let the values of these qubits be $a_j$. Then our $n$-bit approximation for $\theta$ is given by
$$
\theta \approx \sum_{j=0}^n a_j2^{-j}.
$$
We'll perform this conversion from bit values to $\theta$-values and then print the results:

In [ ]:
sim = cirq.Simulator()
result = sim.run(phase_estimator, repetitions=10)

theta_estimates = np.sum(2**np.arange(n_bits)*result.measurements['m'], axis=1)/2**n_bits
print(theta_estimates)

[0.25  0.125 0.25  0.25  0.25  0.25  0.25  0.25  0.25  0.25 ]


When `n_bits` is small, we don't get a very accurate estimate. You can go back and increase `n_bits` and rerun the cells if you like.

##Exercise: As a function

To make experimentation easier, let's pack all this up into a single function that lets us specify $\theta$, the number of bits of of accuracy we want in our approxmation, and the number of repetitions of the algorithm to perform. You could just copy/paste from the previous cells, but it might be a useful exercise to write the whole thing from scratch without peeking.

In [ ]:
def phase_estimation(theta, n_bits, n_reps=10):

  # YOUR CODE HERE
  
  return theta_estimates

### Solution

Here is a solution that just consists of what we did in previous cells all put together.

In [ ]:
def phase_estimation(theta, n_bits, n_reps=10):
  qubits = cirq.LineQubit.range(n_bits)
  u_bit = cirq.NamedQubit('u')

  U = cirq.Z**(2*theta) # Try out a different gate if you like

  phase_estimator = cirq.Circuit()

  phase_estimator.append(cirq.H.on_each(*qubits))
  for i, bit in enumerate(qubits):
    phase_estimator.append(cirq.ControlledGate(U).on(bit,u_bit)**(2**(n_bits-1-i)))
  phase_estimator.append(QFT_inv(n_bits).on(*qubits))
  
  # Measurement gates
  phase_estimator.append(cirq.measure(*qubits, key='m'))

  # Gates to choose initial state for the u_bit. Placing X here chooses the |1> state
  phase_estimator.insert(0,cirq.X(u_bit))

  # Code to simulate measurements
  sim = cirq.Simulator()
  result = sim.run(phase_estimator, repetitions=n_reps)
  
  # Convert measurements into estimates of theta
  theta_estimates = np.sum(2**np.arange(n_bits)*result.measurements['m'], axis=1)/2**n_bits
  
  return theta_estimates

In [ ]:
phase_estimation(0.234, 10)

array([0.22949219, 0.23339844, 0.23339844, 0.234375  , 0.234375  ,
       0.234375  , 0.23339844, 0.23339844, 0.234375  , 0.234375  ])

## Phase Estimation Without an Eigenstate

We can modify our `phase_estimation` method to explore other interesting aspects of the algorithm. Suppose the input to the circuit was not an eigenstate of $U$ at all? You could modify the case we analyzed above by putting an arbitrary single-qubit state on the `u_bit`, or you could replace $Z^{2\theta}$ with something like $X^{2\theta}$ while still passing in a computational basis state. What is the result?

### Solution

Suppose the operator $U$ has eigenstate $|u_j\rangle$ with eigenvalues $e^{2\pi i \theta_j}$. Then in general if we pass in a state of the form
$$
\sum_j \alpha_j|u_j\rangle
$$
then each time we run the circuit we will get an $n$-bit estimate of one of the $\theta_j$ chosen at random, and the probability of choosing a particular $\theta_j$ is given by $|\alpha_j|^2$. One simple test of this is to modify our above code to pass the state
$$
\frac{|0\rangle + |1\rangle}{\sqrt{2}}
$$
into the phase estimator for $Z^{2\theta}$. The state $|1\rangle$ has eigenvalue $e^{2\pi i \theta_j}$ while the state $|0\rangle$ has eigenvalue $1$.

In [ ]:
def phase_estimation(theta, n_bits, n_reps=10):
  qubits = cirq.LineQubit.range(n_bits)
  u_bit = cirq.NamedQubit('u')

  U = cirq.Z**(2*theta)

  phase_estimator = cirq.Circuit()

  phase_estimator.append(cirq.H.on_each(*qubits))
  for i, bit in enumerate(qubits):
    phase_estimator.append(cirq.ControlledGate(U).on(bit,u_bit)**(2**(n_bits-1-i))) # Could have used CZ in this example
  phase_estimator.append(QFT_inv(n_bits).on(*qubits))
  
  phase_estimator.append(cirq.measure(*qubits, key='m'))

  # Changed the X gate here to an H
  phase_estimator.insert(0,cirq.H(u_bit))

  sim = cirq.Simulator()
  result = sim.run(phase_estimator, repetitions=n_reps)
    
  theta_estimates = np.sum(2**np.arange(n_bits)*result.measurements['m'], axis=1)/2**n_bits
  
  return theta_estimates

In [ ]:
phase_estimation(0.234,10)

array([0.234375  , 0.        , 0.        , 0.234375  , 0.234375  ,
       0.23339844, 0.23339844, 0.234375  , 0.234375  , 0.        ])

Notice that half of the measurements yielded the estimate $0$, which corresponds to the eigenvaule $1$. If you experiment with different gates and input states you should see the more general behavior.

Often we won't be able to prepare an exact eigenstate of the operator $U$ we are interested in, so it's very useful to know about this feature of phase estimation. This is crucial for understanding [Shor's algorithm](https://en.wikipedia.org/wiki/Shor%27s_algorithm), for instance.

# Grover's Algorithm

The Grover algorithm takes a black-box oracle implementing a function
$f(x) = 1\text{ if }x=x',~ f(x) = 0 \text{ if } x\neq x'$ and finds $x'$ within a randomly ordered sequence of $N$ items using $O(\sqrt{N}$) operations and $O(N\,  \text{log}N$) gates,
with the probability $p \geq 2/3$.
At the moment, only 2-bit sequences (for which one pass through Grover operator
is enough) are considered.

=== REFERENCE ===

Coles, Eidenbenz et al. Quantum Algorithm Implementations for Beginners
https://arxiv.org/abs/1804.03719

In [ ]:
import cirq

def set_io_qubits(qubit_count):
    """Add the specified number of input and output qubits."""
    input_qubits = [cirq.GridQubit(i, 0) for i in range(qubit_count)]
    output_qubit = cirq.GridQubit(qubit_count, 0)
    #
    return (input_qubits, output_qubit)


def make_oracle(input_qubits, output_qubit, x_bits):
    """Implement function {f(x) = 1 if x==x', f(x) = 0 if x!= x'}."""
    # Make oracle.
    # for (1, 1) it's just a Toffoli gate
    # otherwise negate the zero-bits.
    yield(cirq.X(q) for (q, bit) in zip(input_qubits, x_bits) if not bit)
    yield(cirq.TOFFOLI(input_qubits[0], input_qubits[1], output_qubit))
    yield(cirq.X(q) for (q, bit) in zip(input_qubits, x_bits) if not bit)


def make_grover_circuit(input_qubits, output_qubit, oracle):
    """Find the value recognized by the oracle in sqrt(N) attempts."""
    # For 2 input qubits, that means using Grover operator only once.

    # Initialize qubits.
    yield ([
        cirq.X(output_qubit),
        cirq.H(output_qubit),
        cirq.H.on_each(*input_qubits),
    ])

    # Query oracle.
    yield from oracle

    # Construct Grover operator.
    yield cirq.H.on_each(*input_qubits)
    yield cirq.X.on_each(*input_qubits)
    yield cirq.H.on(input_qubits[1])
    yield cirq.CNOT(input_qubits[0], input_qubits[1])
    yield cirq.H.on(input_qubits[1])
    yield cirq.X.on_each(*input_qubits)
    yield cirq.H.on_each(*input_qubits)

    # Measure the result.
    yield cirq.measure(*input_qubits, key='result')


def bitstring(bits):
    return ''.join(str(int(b)) for b in bits)


qubit_count = 2
circuit_sample_count = 10

#Set up input and output qubits.
(input_qubits, output_qubit) = set_io_qubits(qubit_count)

#Choose the x' and make an oracle which can recognize it.
x_bits = [random.randint(0, 1) for _ in range(qubit_count)]
print('Secret bit sequence: {}'.format(x_bits))

# Make oracle (black box)
oracle = make_oracle(input_qubits, output_qubit, x_bits)

# Embed the oracle into a quantum circuit implementing Grover's algorithm.
# circuit = make_grover_circuit(input_qubits, output_qubit, oracle)
print('Circuit:')
circuit = cirq.Circuit(make_grover_circuit(input_qubits, output_qubit, oracle))
print(circuit)

# Sample from the circuit a couple times.
simulator = cirq.Simulator()
result = simulator.run(circuit, repetitions=circuit_sample_count)

frequencies = result.histogram(key='result', fold_func=bitstring)
print('Sampled results:\n{}'.format(frequencies))

# Check if we actually found the secret value.
most_common_bitstring = frequencies.most_common(1)[0][0]
print('Most common bitstring: {}'.format(most_common_bitstring))
print('Found a match: {}'.format(
    most_common_bitstring == bitstring(x_bits)))


Secret bit sequence: [0, 1]
Circuit:
(0, 0): ───H───X───@───X───H───X───@───X───H───────M('result')───
                   │               │               │
(1, 0): ───H───────@───H───X───H───X───H───X───H───M─────────────
                   │
(2, 0): ───X───H───X─────────────────────────────────────────────
Sampled results:
Counter({'01': 10})
Most common bitstring: 01
Found a match: True


## Grover on the custom Device
Execute Grover's algorithm on the custom Device from the previouse notebook. Use also the optimizers from the other notebook.

**Suggestion**: git clone the newest Cirq repository from github on your machine and develop in a separate Python file.

In [ ]:
# Use something like the following to connect the device to the circuit

# # Make oracle (black box)
# oracle = make_oracle(input_qubits, output_qubit, x_bits)

# # Embed the oracle into a quantum circuit implementing Grover's algorithm.
# # circuit = make_grover_circuit(input_qubits, output_qubit, oracle)
# print('Circuit:')
# circuit = cirq.Circuit(make_grover_circuit(input_qubits, output_qubit, oracle), device=CustomDevice(3,3))
# print(circuit)

# Left to the reader as an exercise

## Quantum Fourier Transform with Unreversed Output

It may be convenient to have an alternative form of the QFT where the output bits are not in the opposite order of the input bits. Can you make an alternative implementation which does that automatically? You may want to consider using SWAP gates.

## Phase Estimation with Arbitrary $U$

Try to implement the phase estimation algorithm in a way that an arbitrary gate $U$ can be supplied and tested. After you've done that, you can test the algorithm on some of your favorite two- or three-qubit gates.

## QFT and Phase Estimation with Adjacency Constraints

Often on a real machine we can't execute two-qubit gates between qubits that are not right next to each other. You'll have noticed that the circuits we defined above involves connections between many different pairs of qubits, which will likely not all be near each other when we try to run the circuit on an actual chip. See if you can modify the examples we went through above in such a way that Cirq validates them for use on the Foxtail or Bristlecone chips. How bit of an example can you handle?